In [ ]:
!python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py train /u/horlavanasta/MasterProject//DataAndExperiments/Experiments/Experiments-1.5T-3T/NNs_Bayesian/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-WeightedCrossEntropy_augmTrue --n_splits 1 --split 0  --batch_size 5               

In [ ]:
!python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py train /u/horlavanasta/MasterProject//DataAndExperiments/Experiments_3-fold/Experiments-1.5T/NNs_Bayesian/SEResNet18Expanded/subject_model-SEResNet18Expanded_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmTrue_20210528_130201/ --resume True --n_splits 3 --nproc 2 --batch_size 8

In [ ]:
def check_history(model_path, epochs, num_folds):
    from visualize.data_utils import get_data_generic
    from argparse import Namespace
    
    finished_folds=0

    if not os.path.exists(os.path.join(model_path, "status.txt")):
        args = Namespace()
        args.model_path=model_path
        args.data_types=["history"]
        args.average_fold=False
        data = get_data_generic(args, reshape_dict=False)
        finished_folds=0
        for fold, fold_data in data["history"].items():
            if len(fold_data[["epoch"]])==epochs:
                finished_folds+=1
        if finished_folds==num_folds:
            print("Finished:", model_path)
            with open(os.path.join(model_path, "status.txt"), "w") as f:
                f.write("Status: job was finished for all folds")
        return finished_folds==num_folds
    else:
        return True
    
def check_results(model_path, MS_list, num_folds):
    import os
    import pathlib
    import numpy as np
    currentDirectory = pathlib.Path(model_path)
    currentPattern = "fold-*"
    flag=True
    for fold_dir in currentDirectory.glob(currentPattern):
        fold = int(str(fold_dir).split("-")[-1])

        selection_metrics = ["best_loss", "best_balanced_accuracy", "last_checkpoint"]
        cnn_classification_dir = os.path.join(model_path, 'fold-%i' % fold, 'cnn_classification')
        
        for selection_metric in selection_metrics:
            modes = ['train', 'validation']
            for ms_el in MS_list:
                modes.append('test_' + ms_el)
                
            for mode in modes:
                if not os.path.exists(os.path.join(cnn_classification_dir, selection_metric,
                                                       '%s_image_level_metrics.tsv' % (mode))):
                    flag=False
                
    return flag
    
def check_complete_test(model_path, num_folds, MS_list):
    import json
    path_params = os.path.join(model_path, "commandline_train.json")
    with open(path_params, "r") as f:
        params = json.load(f)
    epochs=params["epochs"]
#     check_history(model_path, epochs, num_folds)
    return (check_history(model_path, epochs, num_folds) and not check_results(model_path, MS_list, num_folds))
    


In [ ]:
import pathlib
import pandas as pd
import os
import json

folders = []
MS_main_list = ['1.5T', "3T","1.5T-3T"]
MS_list_dict = {'1.5T':['1.5T', '3T'], "3T": ['3T', '1.5T'], "1.5T-3T": ["1.5T-3T"]}
home_folder='/u/horlavanasta/MasterProject/'

isBayesian=True
NBR_BAYESIAN_ITER=10

num_folds=5
for MS in MS_main_list[:]:
#     print("MS %s \n ____________________________________________________________________________________________"%MS)
    model_types = [ "ResNet18",  "ResNet18Expanded", "Conv5_FC3",  "SEResNet50","SEResNet50Expanded" ]
    MS_list = MS_list_dict[MS]
    inference_modes=["mode", "mean"]
    results_folder_general =os.path.join(home_folder, 'Code/ClinicaTools/AD-DL/results/', "Experiments_%d-fold"%num_folds, "Experiments_Bayesian" if isBayesian else "Experiments", 'Experiments-' + MS)
    model_dir_general = os.path.join(home_folder,"DataAndExperiments/Experiments_%d-fold/Experiments-%s"%(num_folds, MS), "NNs_Bayesian" if isBayesian else "NNs")
    CAPS_DIR="$HOME/MasterProject/DataAndExperiments/Data/CAPS"
    if isBayesian:
        NN_FOLDER="NNs_Bayesian"
    else:
        NN_FOLDER="NNs"


    TSV_PATH="/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-%s/labels/test"%MS
    for network in model_types[:]:
        model_dir = os.path.join(model_dir_general, network)
        OUTPUT_DIR="/u/horlavanasta/MasterProject//DataAndExperiments/Experiments_5-fold/Experiments-%s/%s/%s/"%(MS, NN_FOLDER, network)
        POSTFIX="test_%s"%MS
        # output_dir = pathlib.Path(output_dir)
        modelPatter = "subject_model*"
        folders = [f for f in pathlib.Path(model_dir).glob(modelPatter)]

        for f in folders[:]:
            if check_complete_test(f, num_folds=num_folds,MS_list=MS_list):
                print("srun python3 /u/horlavanasta/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py classify %s %s %s %s --bayesian %s --nbr_bayesian_iter %d --selection_metrics balanced_accuracy loss last_checkpoint"%(CAPS_DIR, TSV_PATH, f, POSTFIX, isBayesian, NBR_BAYESIAN_ITER))
                if MS == "1.5T":
                    TEST_MS="3T"
                    TEST_POSTFIX="test_%s"%TEST_MS
                    TEST_TSV_PATH="/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-%s/labels/"%TEST_MS
                    print("srun python3 /u/horlavanasta/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py classify %s %s %s %s --bayesian %s --nbr_bayesian_iter %d --selection_metrics balanced_accuracy loss last_checkpoint"%(CAPS_DIR, TEST_TSV_PATH, f, TEST_POSTFIX, isBayesian, NBR_BAYESIAN_ITER))


                elif MS == "3T":
                    TEST_MS="1.5T"
                    TEST_POSTFIX="test_%s"%TEST_MS
                    TEST_TSV_PATH="/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-%s/labels/"%TEST_MS
                    print("srun python3 /u/horlavanasta/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py classify %s %s %s %s --bayesian %s --nbr_bayesian_iter %d --selection_metrics balanced_accuracy loss last_checkpoint"%(CAPS_DIR, TEST_TSV_PATH, f, TEST_POSTFIX, isBayesian, NBR_BAYESIAN_ITER))

#                     for inference_mode in inference_modes:
#                         results_dir=os.path.join(results_folder_general,network, "%s_inference"%inference_mode)
#                         !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py bayesian $f stat
#                         !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir uncertainty_distribution results history --get_test_from_bayesian True --ba_inference_mode $inference_mode --catplot_type violinplot --uncertainty_metric "total_variance"
#                         !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir uncertainty_distribution results history --average_fold False --get_test_from_bayesian True --ba_inference_mode $inference_mode --catplot_type violinplot --uncertainty_metric "total_variance"



In [4]:
def translate_parameters(args):
    """
    Translate the names of the parameters between command line and source code.
    """
#     args.gpu = True
    args.optimizer = "Adam"
    args.batch_size=8
    args.nproc=8
    # args.loss = "default"

    if hasattr(args, "caps_dir"):
        args.input_dir = args.caps_dir
    if hasattr(args, "unnormalize"):
        args.minmaxnormalization = not args.unnormalize
    if hasattr(args, "slice_direction"):
        args.mri_plane = args.slice_direction
    if hasattr(args, "network_type"):
        args.mode_task = args.network_type

    if not hasattr(args, "selection_threshold"):
        args.selection_threshold = None
        
    if not hasattr(args, "verbose"):
        args.verbose = 0
    if not hasattr(args, "bayesian"):
        args.bayesian = False

    if not hasattr(args, "prepare_dl"):
        if hasattr(args, "use_extracted_features"):
            args.prepare_dl = args.use_extracted_features
        elif hasattr(args, "use_extracted_patches") and args.mode == "patch":
            args.prepare_dl = args.use_extracted_patches
        elif hasattr(args, "use_extracted_slices") and args.mode == "slice":
            args.prepare_dl = args.use_extracted_slices
        elif hasattr(args, "use_extracted_roi") and args.mode == "roi":
            args.prepare_dl = args.use_extracted_roi

    return args

def rename_model(model_path, new_name="SEResNet50"):
    
    from tools.deep_learning.models import init_model
    from tools.deep_learning.iotools import read_json, commandline_to_json
    
    from argparse import Namespace
    from torch.utils.data import DataLoader

    import torchvision.models
    import hiddenlayer as hl
    import torch

    args=Namespace()
#     args.model_path=model_path
    
    args = read_json(args, json_path=os.path.join(model_path, 'commandline.json'))
#     args.model=new_name
    args.output_dir=model_path
    args.nproc=2
    args.batch_size=2


    commandline_to_json(args, filename="commandline.json")
    
    if os.path.exists(os.path.join(args.output_dir, 'commandline_train.json')):
        args=Namespace()
        args = read_json(args, json_path=os.path.join(model_path, 'commandline_train.json'))
#         args.model=new_name
        args.output_dir=model_path
        args.batch_size=8
        args.nproc=8

        commandline_to_json(args, filename="commandline_train.json")
    
#     new_model_folder=model_path.replace(old_name, new_name)
#     os.makedirs(new_model_folder, exist_ok=True)
#     os.rename(model_path,new_model_folder)
#     os.remove(model_path)
#     print(new_model_folder)
    

In [5]:
import pathlib
import pandas as pd
import os
import json

folders = []
MS_main_list = ['1.5T', "3T","1.5T-3T"]
MS_list_dict = {'1.5T':['1.5T', '3T'], "3T": ['3T', '1.5T'], "1.5T-3T": ["1.5T-3T"]}
home_folder='/u/horlavanasta/MasterProject/'

isBayesian=True
for MS in MS_main_list[:]:
    print("MS %s \n ____________________________________________________________________________________________"%MS)
#     model_types = ["SEResNet18", "SEResNet18Expanded"]
    new_model_types = ["SEResNet50", "SEResNet50Expanded"]
    MS_list = MS_list_dict[MS]
    
    model_dir_general = os.path.join(home_folder,"DataAndExperiments/Experiments_5-fold/Experiments-" + MS, "NNs_Bayesian" if isBayesian else "NNs")

    for i, network in enumerate(new_model_types[:]):
        model_dir = os.path.join(model_dir_general, network)
        # output_dir = pathlib.Path(output_dir)
        modelPatter = "subject_model*"
        folders = [f for f in pathlib.Path(model_dir).glob(modelPatter)]

        for f in folders[:]:
            print(str(f))
            rename_model(str(f), new_name=new_model_types[i])
#             show_data(f, plane="sag")
#             show_data(f, plane="cor")
#             show_data(f, plane="axi")
             

MS 1.5T 
 ____________________________________________________________________________________________
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-1.5T/NNs_Bayesian/SEResNet50/subject_model-SEResNet50_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmTrue
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-1.5T/NNs_Bayesian/SEResNet50/subject_model-SEResNet50_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmFalse
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-1.5T/NNs_Bayesian/SEResNet50/subject_model-SEResNet50_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmTrue_20210604_132309
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-1.5T/NNs_Bayesian/SEResNet50/subject_model-SEResNet50_preprocessing-linear_task-AD_CN_norm-1_loss-WeightedCrossEntropy_augmFalse
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-1.5T/N

In [ ]:
def translate_parameters(args):
    """
    Translate the names of the parameters between command line and source code.
    """
    args.gpu = True
    args.optimizer = "Adam"
    args.nproc = 2
    args.batch_size=8
    
    # args.loss = "default"

    if hasattr(args, "caps_dir"):
        args.input_dir = args.caps_dir
    if hasattr(args, "unnormalize"):
        args.minmaxnormalization = not args.unnormalize
    if hasattr(args, "slice_direction"):
        args.mri_plane = args.slice_direction
    if hasattr(args, "network_type"):
        args.mode_task = args.network_type

    if not hasattr(args, "selection_threshold"):
        args.selection_threshold = None
        
    if not hasattr(args, "verbose"):
        args.verbose = 0
    if not hasattr(args, "bayesian"):
        args.bayesian = False

    if not hasattr(args, "prepare_dl"):
        if hasattr(args, "use_extracted_features"):
            args.prepare_dl = args.use_extracted_features
        elif hasattr(args, "use_extracted_patches") and args.mode == "patch":
            args.prepare_dl = args.use_extracted_patches
        elif hasattr(args, "use_extracted_slices") and args.mode == "slice":
            args.prepare_dl = args.use_extracted_slices
        elif hasattr(args, "use_extracted_roi") and args.mode == "roi":
            args.prepare_dl = args.use_extracted_roi

    return args

def show_model(model_folder):
    
    from tools.deep_learning.models import init_model
    from tools.deep_learning.data import (get_transforms,
                                        load_data,
                                        return_dataset,
                                        generate_sampler)
    from tools.deep_learning.iotools import return_logger
    from argparse import Namespace
    from torch.utils.data import DataLoader

    import torchvision.models
    import hiddenlayer as hl
    import torch
    from torch.autograd import Variable

    path_params = os.path.join(model_folder, "commandline.json")
    with open(path_params, "r") as f:
        params = json.load(f)
    params = translate_parameters(Namespace(**params))
    
    main_logger = return_logger(params.verbose, "main process")
    model = init_model(params, initial_shape=None)
    
#     model.to(torch.device('cpu'))

#     import torch.onnx
#     dummy_input = Variable(torch.randn(1,1, 128, 128, 128))
#     torch.onnx.export(model, dummy_input, "/u/horlavanasta/MasterProject/model_%s.onnx"%params.model)
    from torchsummary import summary
    print(model)
    
import pathlib
import pandas as pd
import os
import json

folders = []
MS_main_list = ['1.5T', "3T","1.5T-3T" ]
MS_list_dict = {'1.5T':['1.5T', '3T'], "3T": ['3T', '1.5T'], "1.5T-3T": ["1.5T-3T"]}
home_folder='/u/horlavanasta/MasterProject/'

isBayesian=True
for MS in MS_main_list[:1]:
    print("MS %s \n ____________________________________________________________________________________________"%MS)
    model_types = ["ResNet18"]
    MS_list = MS_list_dict[MS]
    
    model_dir_general = os.path.join(home_folder,"DataAndExperiments/Experiments_5-fold/Experiments-" + MS, "NNs" if isBayesian else "NNs")

    for network in model_types[:]:
        model_dir = os.path.join(model_dir_general, network)
        # output_dir = pathlib.Path(output_dir)
        modelPatter = "subject_model*"
        folders = [f for f in pathlib.Path(model_dir).glob(modelPatter)]

        for f in folders[:1]:
            print(f)
            show_model(f)

In [ ]:
import pathlib
import pandas as pd
import os
import json

folders = []
MS_main_list = ['1.5T', "3T","1.5T-3T" ]
MS_list_dict = {'1.5T':['1.5T', '3T'], "3T": ['3T', '1.5T'], "1.5T-3T": ["1.5T-3T"]}
home_folder='/u/horlavanasta/MasterProject/'

isBayesian=True
for MS in MS_main_list[:1]:
    print("MS %s \n ____________________________________________________________________________________________"%MS)
    model_types = ["ResNet18" ]
    MS_list = MS_list_dict[MS]
    
    model_dir_general = os.path.join(home_folder,"DataAndExperiments/Experiments/Experiments-" + MS, "NNs" if isBayesian else "NNs")

    for network in model_types[:]:
        model_dir = os.path.join(model_dir_general, network)
        # output_dir = pathlib.Path(output_dir)
        modelPatter = "subject_model*"
        folders = [f for f in pathlib.Path(model_dir).glob(modelPatter)]

        for f in folders[:1]:
            print(f)
            show_model(f)
#             show_data(f, plane="sag")
#             show_data(f, plane="cor")
#             show_data(f, plane="axi")
             

In [ ]:
import torchinfo 
